In [ ]:
import numpy as np
import pandas as pd
import os
import csv
import matplotlib.pyplot as plt
import optuna

seed = 1729
np.random.seed(seed)

In [ ]:
import torch
from torch import nn
import torch.optim as optim

torch.manual_seed(seed)
x = torch.rand(5, 3)
print(x)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def create_dataset(data, variable_indexes, lookback_period, step, forecast_period, motif_indexes):
    X1, X2, y = [], [], []  # X1: data, X2: indexes of the motifs, y: distance to the next motif
    
    for idx in range(len(data[0]) - lookback_period - 1):
        if idx % step != 0:
            continue

        window_end_idx = idx + lookback_period
        forecast_period_end = window_end_idx + forecast_period

        # If there are no more matches after the window, break
        if not any([window_end_idx < motif_idx for motif_idx in motif_indexes]):
            break

        # Motif indexes in window, relative to the start of the window
        motif_indexes_in_window = [motif_idx - idx for motif_idx in motif_indexes if idx <= motif_idx <= window_end_idx]
        motif_indexes_in_forecast_period = [motif_idx for motif_idx in motif_indexes if window_end_idx < motif_idx <= forecast_period_end]

        if motif_indexes_in_forecast_period:
            next_match_in_forecast_period = motif_indexes_in_forecast_period[0]
        else:
            next_match_in_forecast_period = -1  # No match in the forecast period but exists in the future

        # Get the data window and transpose to (lookback_period, num_features)
        data_window = data[variable_indexes, idx:window_end_idx].T

        # Calculate `y`
        data_y = -1
        if next_match_in_forecast_period != -1:
            # Index of the next match relative to the end of the window
            data_y = next_match_in_forecast_period - window_end_idx
        
        # Append to lists
        X1.append(torch.tensor(data_window, dtype=torch.float32))  # Now with shape (lookback_period, num_features)
        X2.append(torch.tensor(motif_indexes_in_window, dtype=torch.long)) 
        y.append(data_y) 

    # Pad X2 sequences to have the same length
    X2_padded = pad_sequence(X2, batch_first=True, padding_value=-1)
    
    # Convert lists to torch tensors
    X1 = torch.stack(X1)  # Final shape: (num_samples, lookback_period, num_features)
    y = torch.tensor(y, dtype=torch.float32).unsqueeze(1) 

    return X1, X2_padded, y


In [ ]:
#load data 
n = 100000 #number of data points
k = 3 #number of variables
p = 5 # pattern length
variable_indexes = range(k)

data_scenario3 = np.genfromtxt("../data/syntheticdata/scenario3_n={}_k={}_p={}_min_step={}_max_step={}.csv".format(n, k, p, 5, 45), delimiter=",").astype(int).reshape((k, n))

motif_indexes_scenario3 = np.genfromtxt("../data/syntheticdata/motif_indexes_scenario3_n={}_k={}_p={}_min_step={}_max_step={}.csv".format(n, k, p, 5, 45), delimiter=",").astype(int)

print(motif_indexes_scenario3)


In [ ]:
from timeseries_split import BlockingTimeSeriesSplit

#create index  
indexes = np.arange(len(data_scenario3[0]))

#split data
tscv = BlockingTimeSeriesSplit(n_splits=5)
# Create the figure
fig, ax = plt.subplots(figsize=(10, 6))
for i, (train_index, test_index) in enumerate(tscv.split(indexes)):
    # Plot train and test indices
    ax.plot(train_index, np.zeros_like(train_index) + i, 'o', color='lightblue')
    ax.plot(test_index, np.zeros_like(test_index) + i, 'o', color='red')
    print("TRAIN:", train_index, "TEST:", test_index)
    

ax.set_yticks(np.arange(5), ["Fold {}".format(i) for i in range(1, 6)])
plt.show()

In [ ]:
lookback_period = 100 #window size
step = 5 #step size for the sliding window
forecast_period = 50 #forward window size

#x1: past window, x2: indexes of the motif in the window,  y: next relative index of the motif
X1, X2, y = create_dataset(data_scenario3, variable_indexes, lookback_period, step, forecast_period, motif_indexes_scenario3)

#X1 shape is (num_samples, lookback_period, num_features)
masking_X1 = np.zeros((X1.shape[0], X1.shape[1], 1)) 

for i, obs_motif_indexes in enumerate(X2):
    for j, idx in enumerate(obs_motif_indexes):
        masking_X1[i, idx.item():idx.item()+p, 0] = 1

masking_X1 = torch.tensor(masking_X1, dtype=torch.float32)

# X1, X2, and y are now PyTorch tensors
print("X1 shape:", X1.shape)  # Expected shape: (num_samples, lookback_period, num_features)
print("masking_X1 shape:", masking_X1.shape)  # Expected shape: (num_samples, lookback_period, 1)
print("X2 shape:", X2.shape)  # Expected shape: (num_samples, max_motif_length_in_window)
print("y shape:", y.shape)    # Expected shape: (num_samples, 1)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import optuna
import joblib
from typing import Tuple, List
import csv
from models.lstm_pytorch import LSTMX1Input, LSTMX1_X2BeforeLSTM, LSTMX1_X2AfterLSTM

# Helper classes and functions
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

def evaluate_metrics(predictions: torch.Tensor, targets: torch.Tensor) -> Tuple[float, float]:
    mae = torch.mean(torch.abs(predictions - targets)).item()
    rmse = torch.sqrt(torch.mean((predictions - targets) ** 2)).item()
    return mae, rmse

# Scale training data and apply the scaler to validation and test data
def scale_data(X_train, X_val, X_test):
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train = torch.tensor(scaler.fit_transform(X_train.view(-1, X_train.shape[-1])), dtype=torch.float32).view(X_train.shape)
    X_val = torch.tensor(scaler.transform(X_val.view(-1, X_val.shape[-1])), dtype=torch.float32).view(X_val.shape)
    X_test = torch.tensor(scaler.transform(X_test.view(-1, X_test.shape[-1])), dtype=torch.float32).view(X_test.shape)
    return X_train, X_val, X_test


def train_model_single_input(model, criterion, optimizer, train_loader, val_loader, num_epochs=1000):
    early_stopper = EarlyStopper(patience=10, min_delta=1e-5)
    best_val_loss = float('inf')
    best_model_state = None  # Store the state of the best model

    for epoch in range(num_epochs):
        model.train()
        for batch_X1, batch_y in train_loader:
            batch_X1, batch_y = batch_X1.to(device), batch_y.to(device)
            optimizer.zero_grad()
            loss = criterion(model(batch_X1), batch_y)
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_X1, batch_y in val_loader:
                batch_X1, batch_y = batch_X1.to(device), batch_y.to(device)
                val_loss += criterion(model(batch_X1), batch_y).item()
        avg_val_loss = val_loss / len(val_loader)

        # Check early stopping criteria
        if epoch >= 100 and early_stopper.early_stop(avg_val_loss):
            print(f"Early stopping at epoch {epoch + 1}")
            break

        # Save the best model based on validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()  # Save best model state

    # Load the best model state before returning, to ensure best validation performance
    model.load_state_dict(best_model_state)
    return best_val_loss, model


def train_model_dual_input(model, criterion, optimizer, train_loader, val_loader, num_epochs=1000):
    early_stopper = EarlyStopper(patience=10, min_delta=1e-5)
    best_val_loss = float('inf')
    best_model_state = None  # Store the state of the best model

    for epoch in range(num_epochs):
        model.train()
        for batch_X1, batch_X2, batch_y in train_loader:
            batch_X1, batch_X2, batch_y = batch_X1.to(device), batch_X2.to(device), batch_y.to(device)
            optimizer.zero_grad()
            loss = criterion(model(batch_X1, batch_X2), batch_y)
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_X1, batch_X2, batch_y in val_loader:
                batch_X1, batch_X2, batch_y = batch_X1.to(device), batch_X2.to(device), batch_y.to(device)
                val_loss += criterion(model(batch_X1, batch_X2), batch_y).item()
        avg_val_loss = val_loss / len(val_loader)

        # Check early stopping criteria
        if epoch >= 100 and early_stopper.early_stop(avg_val_loss):
            print(f"Early stopping at epoch {epoch + 1}")
            break

        # Save the best model based on validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()  # Save best model state

    # Load the best model state before returning, to ensure best validation performance
    model.load_state_dict(best_model_state)
    return best_val_loss, model



def run_cross_val(trial, model_class, X1, y, X2=None, criterion=torch.nn.MSELoss()):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    hidden_size = trial.suggest_categorical("hidden_size", [16, 32, 64, 128, 256])
    num_layers = trial.suggest_categorical("num_layers", [1, 2, 3])
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])

    fold_results, test_mae_per_fold, test_rmse_per_fold, test_losses = [], [], [], []  # Added test_losses to track test loss
    best_fold_model = None  # Initialize the best model to track it for return
    best_val_loss = float('inf')  # Track the best validation loss across folds

    for fold, (train_idx, test_idx) in enumerate(BlockingTimeSeriesSplit(n_splits=5).split(X1)):
        train_val_split_idx = int(0.8 * len(train_idx))
        train_idx, val_index = train_idx[:train_val_split_idx], train_idx[train_val_split_idx:]
    
        X1_train, X1_val, X1_test, y_train, y_val, y_test = X1[train_idx], X1[val_index], X1[test_idx], y[train_idx], y[val_index], y[test_idx]
        
        # Prepare train, validation, and test sets
        X1_train_scaled, X1_val_scaled, X1_test_scaled = scale_data(X1_train, X1_val, X1_test)

        if X2 is not None:
            X2_train, X2_val, X2_test = X2[train_idx], X2[val_index], X2[test_idx]

            train_loader = DataLoader(TensorDataset(X1_train_scaled, X2_train, y_train), batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(TensorDataset(X1_val_scaled, X2_val, y_val), batch_size=len(X1_val_scaled), shuffle=False)
            test_loader = DataLoader(TensorDataset(X1_test_scaled, X2_test, y_test), batch_size=len(X1_test_scaled), shuffle=False)
            
            model = model_class(input_size=X1.shape[2], hidden_size=hidden_size, num_layers=num_layers, output_size=1, auxiliary_input_dim = X2.shape[1]).to(device)
            fold_val_loss, model = train_model_dual_input(model, criterion, torch.optim.Adam(model.parameters(), lr=learning_rate), train_loader, val_loader)

        else:
            train_loader = DataLoader(TensorDataset(X1_train_scaled, y_train), batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(TensorDataset(X1_val_scaled, y_val), batch_size=len(X1_val_scaled), shuffle=False)
            test_loader = DataLoader(TensorDataset(X1_test_scaled, y_test), batch_size=len(X1_test_scaled), shuffle=False)
            
            model = model_class(input_size=X1.shape[2], hidden_size=hidden_size, num_layers=num_layers, output_size=1).to(device)
            fold_val_loss, model = train_model_single_input(model, criterion, torch.optim.Adam(model.parameters(), lr=learning_rate), train_loader, val_loader)
        
        fold_results.append(fold_val_loss)

        if fold_val_loss < best_val_loss:
            best_val_loss = fold_val_loss
            best_fold_model = model 

        # Test evaluation
        all_predictions, all_true_values = [], []
        model.eval()
        test_loss = 0  # Initialize test loss for this fold
        with torch.no_grad():
            for batch_data in test_loader:
                if X2 is not None:
                    batch_X1, batch_X2, batch_y = batch_data[0].to(device), batch_data[1].to(device), batch_data[2].to(device)
                    predictions = model(batch_X1, batch_X2)
                else:
                    batch_X1, batch_y = batch_data[0].to(device), batch_data[1].to(device)
                    predictions = model(batch_X1)
                test_loss += criterion(predictions, batch_y).item()  # Accumulate test loss
                all_predictions.append(predictions)
                all_true_values.append(batch_y)

        # Calculate average test loss for the fold
        avg_test_loss = test_loss / len(test_loader)
        test_losses.append(avg_test_loss)  # Append to test_losses for this fold

        all_predictions = torch.cat(all_predictions)
        all_true_values = torch.cat(all_true_values)
        mae, rmse = evaluate_metrics(all_predictions, all_true_values)
        test_mae_per_fold.append(mae)
        test_rmse_per_fold.append(rmse)

    mean_val_loss = np.mean(fold_results)
    mean_test_loss = np.mean(test_losses)  # Calculate mean test loss across folds
    mean_test_mae, std_test_mae = np.mean(test_mae_per_fold), np.std(test_mae_per_fold)
    mean_test_rmse, std_test_rmse = np.mean(test_rmse_per_fold), np.std(test_rmse_per_fold)

    # Log results
    with open(f"results_{model_class.__name__}.csv", mode="a", newline="") as file:
        writer = csv.writer(file)
        if file.tell() == 0:
            writer.writerow([
                "trial_number", "learning_rate", "batch_size", "hidden_size", "num_layers",
                *["fold_{}_val_loss".format(i+1) for i in range(5)], "avg_validation_loss",
                "avg_test_loss",  # Log average test loss
                "test_mae_mean", "test_mae_std", "test_rmse_mean", "test_rmse_std"
            ])
        writer.writerow([
            trial.number, learning_rate, batch_size, hidden_size, num_layers,
            *fold_results, mean_val_loss, mean_test_loss,  # Include mean test loss in log
            mean_test_mae, std_test_mae, mean_test_rmse, std_test_rmse
        ])
        file.flush()

    return mean_val_loss, mean_test_loss, best_fold_model # Return mean test loss along with validation loss


def run_optuna_study(objective_func, model_class, X1, y, file_name: str, n_trials: int = 100, X2=None):
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=seed))
    best_val_loss = float('inf')
    best_model_path = f"{model_class.__name__}_best_model.pth"  # Fixed path for the best model

    def objective(trial):
        criterion = torch.nn.MSELoss()  # Define the criterion here
        trial_val_loss, _, model = objective_func(trial, model_class, X1, y, X2, criterion)  # Pass criterion

        # Save model if it's the best one so far
        nonlocal best_val_loss, best_model_path
        if trial_val_loss < best_val_loss:
            best_val_loss = trial_val_loss

            # Remove previous best model file if it exists
            if os.path.exists(best_model_path):
                os.remove(best_model_path)

            # Save the new best model with a fixed path
            torch.save(model.state_dict(), best_model_path)
            print(f"Saved new best model with validation loss {best_val_loss} at {best_model_path}")

        return trial_val_loss

    # Let Optuna manage trials and pass them to the objective function
    study.optimize(objective, n_trials=n_trials)
    joblib.dump(study, file_name)

    print("Best hyperparameters:", study.best_params)
    print("Best cross-validated validation loss:", study.best_value)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from models.lstm_pytorch import LSTMX1Input
run_optuna_study(run_cross_val, LSTMX1Input, X1, y, "LSTMX1Input_study.pkl", n_trials=10)

In [ ]:
from models.lstm_pytorch import LSTMX1_X2BeforeLSTM
run_optuna_study(run_cross_val, LSTMX1_X2BeforeLSTM, X1, y, "LSTMX1_X2BeforeLSTM_study.pkl", n_trials=10, X2=X2)

In [ ]:
from models.lstm_pytorch import LSTMX1_X2Masking
run_optuna_study(run_cross_val, LSTMX1_X2Masking, X1, y, "LSTMX1_X2Masking_study.pkl", n_trials=10, X2=masking_X1)